# Lobbyradar - Expolratory Data Analysis 

In [78]:
from pymongo import MongoClient
import pprint,sys,re


client = MongoClient()
db = client.lobbyradar
entity_cursor = db.entities.find()


In [79]:
# remove unicode prefix, taken from:
# http://stackoverflow.com/questions/16888409/suppress-unicode-prefix-on-strings-when-using-pprint
class writer :
    def write(self, text):
        text=re.sub(r'u\'([^\']*)\'', r'\1',text)
        sys.stdout.write(text)

wrt=writer()
p = pprint.PrettyPrinter(stream=wrt)

The Entities document contains approx. 20k persons and 6.5k non-persons, which are refered to as "entities":

In [80]:
 p.pprint(db.entities.distinct("type"))

[entity, person]


In [81]:
from bson.son import SON
pipeline = [ {"$group": {"_id": "$type", "counter": {"$sum": 1}}}]
 
p.pprint(list(db.entities.aggregate(pipeline, cursor={})))

[{_id: entity, counter: 6424}, {_id: person, counter: 19534}]


Durch Anzeige aller Relationentypen werden folgende Eigenschaften der bereitgestellten Informationen deutlich:
1. Die Sprache der erfassten Daten ist nicht konsistent, z.B.: bei "subsidiary" und "Tochterfirma", wobei beide Begriffe die gleiche bedeuting haben.
2. Manche Daten sind mit Schrieb- bzw. Typfehler erfasst, z.b.: "ececutive" und "subisdiary".
3. In manchen Fällen ist es nicht vollkommen klar, was unter die Beziehungstyp zu verstehen ist und es Bedarf weitere Recherche, z.B.: bei "publication" und "government".

In [82]:
p.pprint(db.relations.distinct('type'))

[Bundesdatenschutzbeauftragte,
 Hausausweise,
 Mitglied,
 Position,
 Tochterfirma,
 Vorsitzender,
 activity,
 association,
 business,
 committee,
 consulting,
 donation,
 ececutive,
 executive,
 government,
 lobbyist,
 member,
 mitglied,
 position,
 publication,
 sponsoring,
 subisdiary,
 subsidiary]


Möchte man mit diesen Typ-Kategorien arbeiten, sollte man Ausreißer eliminieren oder mit andern Kategorien vereinigen:

In [83]:
types = list(db.relations.aggregate(pipeline, cursor={}))
for typ in types:
    if typ["counter"] < 5:
        p.pprint(typ)

{_id: publication, counter: 1}
{_id: ececutive, counter: 2}
{_id: lobbyist, counter: 1}
{_id: Bundesdatenschutzbeauftragte, counter: 1}
{_id: sponsoring, counter: 3}
{_id: Tochterfirma, counter: 4}
{_id: Vorsitzender, counter: 1}
{_id: subisdiary, counter: 2}


Sowohl Entitäte, als auch Relationen verfügen über beschreibungs-"tags":

In [84]:
print "All relations tags:"
p.pprint(db.relations.distinct("tags"))

All relations tags:
[None,
 anwaltskanzlei,
 aufsichtsrat,
 ausland,
 auto,
 bank,
 bertelsmann,
 beteiligungsgesellschaft,
 bildung,
 bundeswehr,
 cdu,
 chemie,
 committee,
 csu,
 dax,
 digitalisierung,
 eisenbahn,
 enenrgie,
 energie,
 experte,
 fdp,
 fernsehen,
 fernsehrat,
 gazprom,
 gesundheit,
 grune,
 handel,
 hausausweise,
 immobilien,
 industrie,
 internet,
 it,
 kabinett,
 kabinette,
 kernkraft,
 krankenkasse,
 laender,
 landwirtschaft,
 lebensmittel,
 lobbyist,
 luftfahrt,
 luftverkehr,
 mdb,
 medien,
 naher-osten,
 nebentaetigkeit,
 oel,
 oepp,
 oesterreich,
 partei,
 parteilos,
 parteispende,
 personaldienstleister,
 pharma,
 politik,
 pr-agentur,
 pr-und-lobbyagentur,
 regierung,
 ruestung,
 schule,
 seefahrt,
 seitenwechsler,
 solar,
 spd,
 spende,
 stahl,
 steinkohle,
 stiftung,
 telekommunikation,
 terrorismus,
 think-tank,
 ukraine,
 umwelt,
 verbraucherschutz,
 verkehr,
 versicherung,
 vorstand,
 wasser,
 werbung]


In [85]:
 print "All entity tags:"
p.pprint(db.entities.distinct("tags"))

All entity tags:
[None,
 Architektur,
 Auto / Verkehr,
 Bank,
 Banken / Versicherungen,
 Bau,
 Beton,
 Bundestagsfraktion,
 Bundeswehr,
 Chemie,
 Energie / Wasser / Versorgung,
 Finanzen,
 Gesundheit,
 Gewerkschaft,
 Handel,
 Holz,
 IT,
 Immobilien,
 Industrie,
 Lebensmittel,
 Medien,
 Medien / IT,
 Medizin,
 Metall,
 M\xf6bel,
 Public Affairs und Lobbyagentur,
 Religion,
 R\xfcstung,
 Sport,
 Stahl,
 Umwelt,
 Verlag,
 Versandhandel,
 Verwaltung / Politik,
 Werbung / PR,
 Zement,
 advokatisch,
 akademisch,
 alkohol,
 anwaltskanzlei,
 architektur,
 aufsichtsrat,
 auto,
 baden-wurttemberg,
 bank,
 banken,
 bau,
 bayern,
 bergbau,
 berlin,
 bertelsmann,
 beteiligungsgesellschaft,
 bildung,
 brandenburg,
 bremen,
 bundeskanzlerin,
 bundesregierung,
 bundestag,
 bundestagsfraktion,
 bundeswehr,
 cdu,
 chemie,
 committee,
 consultating,
 consulting,
 csu,
 dax,
 eisenbahn,
 elektronik,
 eletronik,
 enenrgie,
 energie,
 executive,
 experte,
 fdp,
 fernsehen,
 film,
 finanzdienstleistungen,
 f

In [86]:
query = {'type':'entity', 'importer': {'$in':['parteispenden','parteispenden13', 'parteispenden14']}}
projection = {'_id':0, 'name':1} 
for i in db.entities.find(query, projection):
    p.pprint(i)

{name: Aesculap AG}
{name: BHW Bausparkasse AG}
{name: Franz Haniel & Cie GmbH}
{name: Michael Bob GmbH}
{name: Pfeifer & Langen KG}
{name: Turbiner-Veszkow-Wollh\xe4ndler GbR}
{name: Agricola Verwaltungsges KG}
{name: Audi AG}
{name: Derag, Deutsche Realbesitz AG & Co KG}
{name: Duerr & Partner Beteiligungsges mbH}
{name: Kneipp-Werke OHG}
{name: Kommanditgesellschaft Allgemeine Leasing GmbH & Co}
{name: Mac Mode GmbH & Co. KGaA}
{name: B\xfcll & Liedtke, Projekt Entwicklung GmbH}
{name: Central Krankenversicherung AG}
{name: Papenburg GmbH}
{name: Peter Gro\xdf GmbH}
{name: Peter Waldvogel (Fa)}
{name: Arbeitgeberverband der hessischen Metallindustrie Bezirksgruppe Nordhessen e.V.}
{name: ABB Asea Brown Boveri AG}
{name: DeTeWe}
{name: Deutsche Babcock AG}
{name: Verband der Nord-Westdt.Textil- u. Bekleidungsindustrie e. V.}
{name: Wittgensteiner Kliniken AG}
{name: Spielbank Berlin GmbH}
{name: Assekurranz Schirmer und Grundheber GmbH}
{name: LSV L\xf6blein Service- und Verwaltungs-

In [87]:
pipeline = [ {"$group": {"_id": "$tags", "count": {"$sum": 1}}} , {"$sort": {"count":-1} }]
p.pprint(list(db.relations.aggregate(pipeline, cursor={})))

[{_id: [], count: 17450},
 {_id: [partei], count: 2753},
 {_id: [nebentaetigkeit], count: 2527},
 {_id: [committee], count: 1683},
 {_id: [pr-und-lobbyagentur], count: 1121},
 {_id: [anwaltskanzlei], count: 1051},
 {_id: [partei, spende, parteispende], count: 987},
 {_id: [mdb], count: 629},
 {_id: [laender], count: 503},
 {_id: [dax, aufsichtsrat], count: 414},
 {_id: [pharma], count: 279},
 {_id: [kabinette, kabinett, regierung], count: 254},
 {_id: [kabinette], count: 227},
 {_id: [dax, vorstand], count: 157},
 {_id: [ruestung], count: 110},
 {_id: [bank], count: 78},
 {_id: [committee, ruestung], count: 63},
 {_id: [kabinette, partei], count: 62},
 {_id: [partei, spende, parteispende, laender], count: 58},
 {_id: [committee, bank], count: 58},
 {_id: [pr-und-lobbyagentur, pharma], count: 51},
 {_id: [dax, aufsichtsrat, pharma], count: 51},
 {_id: [seitenwechsler, spd], count: 37},
 {_id: [seitenwechsler, cdu], count: 36},
 {_id: [lebensmittel], count: 35},
 {_id: [pharma, gesundhei

Während die Relationen nur eine Beziehung des Typs "lobbyist" haben, kann mann viele Personen und Organisationen in der entities Collection finden, die als Lobbyisten bezeichnet werden:

In [88]:
print db.entities.find({"tags" :{"$in": [ "lobbyismus", "lobbyist", "lobbyorganisation"]}}).count()

17001


Ebenso sind zahlreiche Beispiele für Spender zu finden, allerdings nicht im Bezug auf einer Organisation, sondern als gernelle Bezeichnung:

In [89]:
print db.entities.find({"tags" :{"$in": [ "parteispenden", "parteispenden13", "parteispenden14"]}}).count()

3538


In [90]:
def display(query, projection):
    data = db.entities.find(query, projection)
    data = data.sort("name") 
    for i in data:
        p.pprint(i)

In [91]:
def entities_find(key, value):
    query = {key: value}
    projection = {"_id":0, "name":1}
    display(query, projection)
   
    

In [92]:
entities_find("tags", "IT")

{name: ArGe PERSER Arbeitsgemeinschaft der Personalabrechnungs-Software-Ersteller}
{name: Arbeitskreis Amateurfunk und Telekommunikation in der Schule e.V. (AATiS)}
{name: BSA Business Software Alliance}
{name: Bundesverband Deutsche Startups e.V.}
{name: Bundesverband IT-Mittelstand e.V. (BITMi)}
{name: Bundesverband Informationswirtschaft, Telekommunikation und neue Medien (BITKOM) e.V.}
{name: Bundesverband Internetmedizin e.V.}
{name: Chaos Computer Club e.V. - CCC -}
{name: Computacenter}
{name: Deutsche Telekom}
{name: Deutscher Verband f\xfcr Telekommunikation und Medien e.V.}
{name: Enquete-Kommission Internet und digitale Gesellschaft}
{name: Fujitsu Computer GmbH}
{name: I.D.I. Interessenverband Deutsches Internet. e.V.}
{name: IT-Gr\xfcnderzentrum GmbH}
{name: IT-Initiative Mecklenburg-Vorpommern e.V.}
{name: Interactive Software Federation of Europe}
{name: J Dreger IT-Operations}
{name: NextWave Telecom}
{name: Rocket Internet}
{name: SAP}
{name: Software AG}
{name: TeleTr

Merkwürdigerweise findet man Bundeskanzlerin Merkel nicht, wenn man nach den cdu-tag sucht: 

In [93]:
query = {"type": "person", "tags": "linke"}
projection = {"_id":0, "name":1}
display(query, projection)


In [40]:
query = {"name":"Angela Merkel"}
projection = {"_id":0, "tags":1}
display(query, projection)


{tags: [kabinette,
           kabinett-merkel-i,
           verwaltung,
           politik,
           kabinett-merkel-ii,
           kabinett-merkel-iii,
           mdb,
           bundestag,
           parteispenden,
           bundeskanzlerin]}


In [41]:
tmp = db.entities.find_one({"name":"Angela Merkel"})
p.pprint(tmp)

# vornamen: merkel, nachnamen: angela ..hemm

{_id: ObjectId('54c2a4b4fe6a42c82bbaafac'),
 aliases: [Dr. Angela Merkel, Angela Merkel],
 created: datetime.datetime(2015, 1, 23, 19, 44, 50, 772000),
 data: [{auto: True,
            created: datetime.datetime(2015, 1, 30, 13, 16, 19, 123000),
            desc: Quelle,
            format: link,
            id: 9dc681ff88ef78470f58dad0a22fde397147a9f46b416478d2fd87233fa728ab,
            key: source,
            updated: datetime.datetime(2015, 1, 30, 13, 16, 19, 123000),
            value: {remark: created by bundestag importer,
                       url: http://www.bundestag.de/bundestag/abgeordnete18/biografien/M/merkel_angela/258788}},
           {auto: True,
            created: datetime.datetime(2015, 1, 30, 13, 16, 19, 123000),
            desc: Vornamen,
            format: string,
            id: dfdee4542498db7db194d7f76a2b7912d5de8efa322e97eb49c7bee72028fe7a,
            key: names,
            updated: datetime.datetime(2015, 3, 1, 20, 26, 4, 833000),
            value: M

Möchte man die tags als Indikator benutzen, stellt sich die Frage, ob alle Entitäten tags haben:

In [42]:
db.entities.find({"tags": []}).count()

1734

Bei Parteien stellt sich die Frage, wie es zu erkennen ist, dass es sich um eine Partei handelt:

In [43]:
db.entities.find_one({"name": "NPD"})

{u'_id': ObjectId('55675a3fc4b1b4847227be3c'),
 u'aliases': [],
 u'created': datetime.datetime(2015, 5, 28, 18, 11, 8, 36000),
 u'data': [{u'auto': True,
   u'created': datetime.datetime(2015, 5, 28, 18, 11, 8, 54000),
   u'desc': u'Partei',
   u'format': u'string',
   u'id': u'82554fd8eb62b7bb912e206ce4ccda2f940b7f914006c7801e7e6a67e72b075e',
   u'key': u'partei',
   u'updated': datetime.datetime(2015, 5, 28, 18, 11, 8, 54000),
   u'value': u'NPD'}],
 u'importer': u'parteispenden13',
 u'name': u'NPD',
 u'search': [u'npd'],
 u'slug': u'npd',
 u'tags': [u'partei'],
 u'type': u'entity',
 u'updated': datetime.datetime(2015, 5, 28, 18, 11, 11, 937000)}

In [44]:
db.entities.find_one({"name": "Fraktion Die Linke"})


{u'_id': ObjectId('54c71e95349d25992bca5370'),
 u'aliases': [u'Linksfraktion', u'Linke Fraktion'],
 u'created': datetime.datetime(2015, 1, 27, 5, 13, 52, 835000),
 u'data': [{u'auto': True,
   u'created': datetime.datetime(2015, 1, 30, 13, 16, 19, 62000),
   u'desc': u'Partei',
   u'format': u'string',
   u'id': u'9b3a49aa87efb844cc781e931086783a55b7e6fa9aff0de12270bd985cdff5f7',
   u'key': u'partei',
   u'updated': datetime.datetime(2015, 1, 30, 13, 16, 19, 62000),
   u'value': u'Die Linke'}],
 u'importer': u'bundestag',
 u'name': u'Fraktion Die Linke',
 u'search': [u'fraktion die linke', u'linksfraktion', u'linke fraktion'],
 u'slug': u'fraktion die linke',
 u'tags': [u'linke', u'Bundestagsfraktion'],
 u'type': u'entity',
 u'updated': datetime.datetime(2015, 1, 30, 13, 16, 51, 446000)}

In [45]:
db.entities.find_one({"name": "SPD-Bundestagsfraktion"})


{u'_id': ObjectId('54c71e95349d25992bca5385'),
 u'aliases': [u'SPD-Bundestagsfraktion',
  u'SPD-Fraktion',
  u'Sozialdemokratische Fraktion'],
 u'created': datetime.datetime(2015, 1, 27, 5, 13, 52, 835000),
 u'data': [{u'auto': True,
   u'created': datetime.datetime(2015, 4, 16, 18, 5, 1, 374000),
   u'desc': u'Quelle',
   u'format': u'link',
   u'id': u'0bd9a93ba337f1decd6affc297a2763360a7c619f77faf39678022f8962087b7',
   u'key': u'source',
   u'updated': datetime.datetime(2015, 4, 16, 18, 5, 1, 374000),
   u'value': {u'remark': u'created by seitenwechsler importer',
    u'url': u'https://lobbypedia.de/wiki/Seitenwechsler_in_Deutschland_im_%C3%9Cberblick'}},
  {u'auto': True,
   u'created': datetime.datetime(2015, 1, 30, 13, 16, 19, 62000),
   u'desc': u'Partei',
   u'format': u'string',
   u'id': u'3fc1195a02cc09a6e34983f9765950f47d9ebdda257e784466a18261c310b0f2',
   u'key': u'partei',
   u'updated': datetime.datetime(2015, 1, 30, 13, 16, 19, 62000),
   u'value': u'SPD'}],
 u'importe